In [ ]:
import pandas as pd

dados = {
    'Id': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'Sexo': ['F', 'F', 'M', 'F', 'M'],
    'Cirurgia?': ['N', 'N', 'S', 'N', 'N'],
    'TSH': [6.8, 7.3, 8.8, 6.9, 5.4],
    'TT4': [156.2, 152.9, 148.4, 132.7, 150.9],
    'TI': ['N', 'N', 'S', 'N', 'N'],
    'Diagnóstico': ['Neg', 'Neg', 'HipoPrim', 'HipoComp', 'Neg']
}

df = pd.DataFrame(dados)

dados_quali = lambda x: pd.Categorical(df[x].astype(str)).codes
for coluna in ['Sexo', 'Cirurgia?', 'TI', 'Diagnóstico']:
    df[coluna] = pd.Categorical(df[coluna]).codes

dados_normalizados = lambda x: (df[x] - df[x].min()) / (df[x].max() - df[x].min())
for coluna in ['TSH', 'TT4']:
    df[coluna] = dados_normalizados(coluna)

euclidian_dist = lambda x, y: ((df[x] - df[y]) ** 2).sum() ** 0.5

euclidian_dist(df.columns[1], df.columns[2])
euclidian_dist(df.columns[1], df.columns[3])
euclidian_dist(df.columns[1], df.columns[4])
euclidian_dist(df.columns[1], df.columns[5])
euclidian_dist(df.columns[1], df.columns[6])



In [11]:

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from collections import Counter

# Dataset original
dados = {
    'Id': ['P1', 'P2', 'P3', 'P4', 'P5'],
    'Sexo': ['F', 'F', 'M', 'F', 'M'],
    'Cirurgia?': ['N', 'N', 'S', 'N', 'N'],
    'TSH': [6.8, 7.3, 8.8, 6.9, 5.4],
    'TT4': [156.2, 152.9, 148.4, 132.7, 150.9],
    'TI': ['N', 'N', 'S', 'N', 'N'],
    'Diagnóstico': ['Neg', 'Neg', 'HipoPrim', 'HipoComp', 'Neg']
}

df = pd.DataFrame(dados)

# Codificar categorias
for coluna in ['Sexo', 'Cirurgia?', 'TI', 'Diagnóstico']:
    df[coluna] = pd.Categorical(df[coluna])
    df[coluna + '_code'] = df[coluna].cat.codes

# Normalizar TSH e TT4
scaler = MinMaxScaler()
df[['TSH_norm', 'TT4_norm']] = scaler.fit_transform(df[['TSH', 'TT4']])

# Novo paciente (atributos codificados conforme as categorias originais)
novo = {
    'Sexo_code': df['Sexo'].cat.categories.get_loc('F'),
    'Cirurgia?_code': df['Cirurgia?'].cat.categories.get_loc('N'),
    'TI_code': df['TI'].cat.categories.get_loc('S'),
    'TSH_norm': scaler.transform([[7.0, 150.0]])[0][0],
    'TT4_norm': scaler.transform([[7.0, 150.0]])[0][1]
}

# Distância euclidiana com base nos atributos relevantes
atributos = ['Sexo_code', 'Cirurgia?_code', 'TSH_norm', 'TT4_norm', 'TI_code']

df['distância'] = df.apply(lambda row: euclidean(
    [row[a] for a in atributos],
    [novo[a] for a in atributos]
), axis=1)

# Classificar para diferentes valores de k
def classificar(k):
    vizinhos = df.nsmallest(k, 'distância')
    mais_comum = Counter(vizinhos['Diagnóstico']).most_common(1)[0][0]
    return mais_comum

print("Classificação para k=1:", classificar(1))
print("Classificação para k=3:", classificar(3))
print("Classificação para k=5:", classificar(5))

Classificação para k=1: Neg
Classificação para k=3: Neg
Classificação para k=5: Neg


c:\Users\Pablo Monteiro\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Pablo Monteiro\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [12]:
df

,Id,Sexo,Cirurgia?,TSH,TT4,TI,Diagnóstico,Sexo_code,Cirurgia?_code,TI_code,Diagnóstico_code,TSH_norm,TT4_norm,distância
0,P1,F,N,6.8,156.2,N,Neg,0,0,0,2,0.411765,1.000000,1.035889
1,P2,F,N,7.3,152.9,N,Neg,0,0,0,2,0.558824,0.859574,1.011442
2,P3,M,S,8.8,148.4,S,HipoPrim,1,1,1,1,1.000000,0.668085,1.511593
3,P4,F,N,6.9,132.7,N,HipoComp,0,0,0,0,0.441176,0.000000,1.242100
4,P5,M,N,5.4,150.9,N,Neg,1,0,0,2,0.000000,0.774468,1.490946
